In [1]:
#import cadquery_bundled
import cadquery as cq
import numpy as np
import math
import matplotlib.pyplot as plt

In [7]:
BASE_PARAMETERS = {}
BASE_PARAMETERS["thickness"] = 4
BASE_PARAMETERS["extrusion_distance"] = 8
BASE_PARAMETERS["length"] = 12
BASE_PARAMETERS["strain"] = 0.04
BASE_PARAMETERS["nose_angle"] = 85
BASE_PARAMETERS["name"] = "default_cantilever"
BASE_PARAMETERS["r_bot"] = 1

def create_cantilever(modified_parameters=dict()):
    p = BASE_PARAMETERS.copy()
    p.update(modified_parameters)

    # Multiplying all values by 10 to fix weirdness
    th = p["thickness"]
    l = p["length"]
    strain = p["strain"]
    nose_angle = math.radians(p["nose_angle"])
    fatness = p["extrusion_distance"]
    name = p["name"]


    arm_length = l

    nose_height = 1.09 * strain * arm_length ** 2 / th
    nose_x = nose_height / math.tan(nose_angle)

    # Point coordinates no radius business
    p_c = [(0, 0),  # Start
           (l * 1.20 + nose_x, 1 / 2 * th * 1.25),
           (l * 1.20 + nose_x, 3 / 4 * th),
           (l * 1.07 + nose_x, th + nose_height),
           (l + nose_x, th + nose_height),
           (l, th),
           (0, th)]

    sketch = (
        cq.Workplane("XY")
    )

    data = np.array(p_c)

    x = data[:, 0]
    y = data[:, 1]

    sketch = sketch.moveTo(x[0], y[0])
    for i in range(1, len(x)):
        x1, y1 = x[i], y[i]
        sketch = sketch.lineTo(x1, y1)

    sketch = sketch.close()
    sketch = sketch.extrude(width)

    assy = cq.Assembly()
    assy.add(sketch, name="sketch", color=cq.Color("gray"))
    cq.exporters.export(sketch, name + ".step")

    # Open stepfile to edit
    from steputils import p21
    fname = name + ".step"
    stepfile = p21.readfile(fname)

    product = stepfile.data[0].get("#7")
    product.entity.params = (name,)

    stepfile.save(fname)
    return sketch

def create_cantilever_with_bot_rad(modified_parameters=dict()):
    p = BASE_PARAMETERS.copy()
    p.update(modified_parameters)

    # Multiplying all values by 10 to fix weirdness
    th = p["thickness"]
    l = p["length"]
    strain = p["strain"]
    nose_angle = math.radians(p["nose_angle"])
    fatness = p["extrusion_distance"]
    r_bot = p["r_bot"]
    r_bot = 1.5
    name = p["name"]
    
    arm_length = l

    nose_height = 1.09 * strain * arm_length ** 2 / th
    nose_x = nose_height / math.tan(nose_angle)


    
    bot_radius_sweep_angle = math.atan(l / (th / 2))
    sin_th = math.sin(bot_radius_sweep_angle)
    cos_th = math.cos(bot_radius_sweep_angle)
    x_rad = (1 - cos_th) * r_bot  # The x-length of bot radius arc
    y_rad = sin_th * r_bot -r_bot  # The y-length of bot radius arc
    
    x_org = r_bot
    y_org = -r_bot
    deg = math.radians(2)
    point_x = x_org - r_bot*math.cos(deg)
    point_y = y_org + r_bot*math.sin(deg)
    
    # Start by making bottom arc
    p2 = (x_rad, y_rad) 
   
    p1 = (point_x, point_y)
    p0 = (0, -r_bot)
    
    part = (
        cq.Workplane("XY")
        .moveTo(p0[0], p0[1])
        .threePointArc(p1,p2)

    )
   

    
    # Point coordinates no radius business
    p_c = [p2,  # Start
           (l * 1.20 + nose_x, 1 / 2 * th * 1.25),
           (l * 1.20 + nose_x, 3 / 4 * th),
           (l * 1.07 + nose_x, th + nose_height),
           (l + nose_x, th + nose_height),
           (l, th),
           (0, th),
           p1
          ]

    data = np.array(p_c)

    x = data[:, 0]
    y = data[:, 1]

    part = part.moveTo(x[0], y[0])
    for i in range(1, len(x)):
        x1, y1 = x[i], y[i]
        part = part.lineTo(x1, y1)

    part = part.close()
    part = part.extrude(fatness)

    # Open stepfile to edit
    from steputils import p21
    fname = name + ".step"
    stepfile = p21.readfile(fname)

    product = stepfile.data[0].get("#7")
    product.entity.params = (name,)

    stepfile.save(fname)
    plt.plot(x,y)
    return part


def create_cantilever_with_top_rad(modified_parameters=dict()):
    p = BASE_PARAMETERS.copy()
    p.update(modified_parameters)

    # Multiplying all values by 10 to fix weirdness
    th = p["thickness"]
    l = p["length"]
    strain = p["strain"]
    nose_angle = math.radians(p["nose_angle"])
    fatness = p["extrusion_distance"]
    r_bot = p["r_bot"]
    r_bot = 1.5
    name = p["name"]

    th = 8
    
    arm_length = l

    nose_height = 1.09 * strain * arm_length ** 2 / th
    nose_x = nose_height / math.tan(nose_angle)

    shift_z = th/2
    part = (
        cq.Workplane("XY")
        .translate((0,0,shift_z))
    )
   
    # Point coordinates no radius business
    p_c = [(0,0),  # Starts at bottom
           (l * 1.20 + nose_x, 1 / 2 * th * 1.25),
           (l * 1.20 + nose_x, 3 / 4 * th),
           (l * 1.07 + nose_x, th + nose_height),
           (l + nose_x, th + nose_height),
           (l, th),
           (r_bot, th)
          ]

    data = np.array(p_c)

    x = data[:, 0]
    y = data[:, 1]

    part = part.moveTo(x[0], y[0])
    for i in range(1, len(x)):
        x1, y1 = x[i], y[i]
        part = part.lineTo(x1, y1)

    bot_radius_sweep_angle = math.atan(l / (th / 2))
    sin_th = math.sin(bot_radius_sweep_angle)
    cos_th = math.cos(bot_radius_sweep_angle)
    x_rad = (1 - cos_th) * r_bot  # The x-length of bot radius arc
    y_rad = sin_th * r_bot -r_bot  # The y-length of bot radius arc
    
    x_org = r_bot
    y_org = th + r_bot
    deg = math.radians(2)
    point_x = x_org - r_bot*math.cos(deg)
    point_y = y_org + r_bot*math.sin(deg)


    # Top arc
    
    p0 = (0, th + r_bot)
    p1 = (r_bot, th)
    p2 = (r_bot - 0.1, th)
    
    part = part.threePointArc(p2,p0)

    part = part.close()
    part = part.extrude(fatness/2, both=True)
    part = part.translate((0,-th/2,0))
      
    cq.exporters.export(part, name + ".step")
    
    return part


In [167]:
BASE_PARAMETERS = {}
BASE_PARAMETERS["thickness"] = 4
BASE_PARAMETERS["extrusion_distance"] = 8
BASE_PARAMETERS["length"] = 12
BASE_PARAMETERS["strain"] = 0.04
BASE_PARAMETERS["nose_angle"] = math.radians(85)
BASE_PARAMETERS["name"] = "default_cantilever"
BASE_PARAMETERS["r_top"] = 1.5

def create_cantilever_with_top_rad(modified_parameters=dict()):
    # Step 1: Set variables to parameters
    p = BASE_PARAMETERS.copy()
    p.update(modified_parameters)

    relevant_params = ["thickness", "length", "strain", "nose_angle",
                       "extrusion_distance", "r_top", "name"]

    th,l,strain,nose_angle,width,r_top,name = [p[x] for x in relevant_params]
    
    nose_height = 1.09 * strain * l ** 2 / th
    nose_x = nose_height / math.tan(nose_angle)

    # Step 2: Draw straight lines
    point_data = np.array([(0,0),  # Starts at bottom
           (l * 1.20 + nose_x, 1 / 2 * th * 1.25),
           (l * 1.20 + nose_x, 3 / 4 * th),
           (l * 1.07 + nose_x, th + nose_height),
           (l + nose_x, th + nose_height),
           (l, th),
           (r_top, th)
          ])
    x,y = point_data[:, 0], point_data[:,1]

    part = (
        cq.Workplane("XY")
        .moveTo(x[0], y[0])
    )    
    for i in range(1, len(x)):
        part = part.lineTo(x[i], y[i])

    # Step 3: Create a radius arc
    #An arc is drawn from p0 to p2
    # To get the correct arc, an additional point (p1) must be identified
    p0 = (x[-1], y[-1])
    p2 = (0, th + r_top) # Top of radius

    deg = math.radians(10) # An arbitrary angle on the arc
    p1_x = r_top*(1 - math.sin(deg)) 
    p1_y = r_top*(1 - math.cos(deg)) + th
    p1 = (p1_x, p1_y)
    part = part.threePointArc(p1,p2)
    
    # Step 4: close the curve and extrude
    part = part.close()
    part = part.extrude(width/2, both=True)

    # Step 4: Position part so that origin is in the intended center of part
    part = part.translate((0,-th/2,0))

    # Step 5: Export file
    cq.exporters.export(part, name + ".step")
    return part


In [168]:
cant = create_cantilever_with_top_rad()
cant

In [24]:
from cadquery import Vector
a = [1,1]
b = [1,5]
c = Vector([2,5])
result = (
    cq.Workplane("XY")
    .threePointArc(a,b)
    .close()

)

display(result)

In [70]:
result = (
    cq.Workplane()
    .box(5, 5, 1)
    .faces(">Z")
    .sketch()
    .arc((0.6, 0.6), 0.4, 0.0, 360.0)
    .close()
    .finalize()
    #.regularPolygon(2, 3, tag="outer")
    #.regularPolygon(1.5, 3, mode="s")
    #.vertices(tag="outer")
    #.fillet(0.2)
    #.finalize()
    #.extrude(0.5)
)
result.vertices.__doc__

'\n        Select the vertices of objects on the stack, optionally filtering the selection. If there\n        are multiple objects on the stack, the vertices of all objects are collected and a list of\n        all the distinct vertices is returned.\n\n        :param selector: optional Selector object, or string selector expression\n            (see :class:`StringSyntaxSelector`)\n        :param tag: if set, search the tagged object instead of self\n        :return: a CQ object who\'s stack contains  the *distinct* vertices of *all* objects on the\n            current stack, after being filtered by the selector, if provided\n\n        If there are no vertices for any objects on the current stack, an empty CQ object\n        is returned\n\n        The typical use is to select the vertices of a single object on the stack. For example::\n\n            Workplane().box(1, 1, 1).faces("+Z").vertices().size()\n\n        returns 4, because the topmost face of a cube will contain four vertices. 

hello


In [34]:
BASE_PARAMETERS = {}
BASE_PARAMETERS["thickness"] = 4
BASE_PARAMETERS["width"] = 8
BASE_PARAMETERS["length"] = 12
BASE_PARAMETERS["strain"] = 0.05
BASE_PARAMETERS["nose_angle"] = math.radians(60)
BASE_PARAMETERS["name"] = "default_cantilever"
BASE_PARAMETERS["r_top"] = 1.5

# Step 1: Set variables to parameters
p = BASE_PARAMETERS.copy()


relevant_params = ["thickness", "length", "strain", "nose_angle",
                   "width", "r_top", "name"]

th, l, strain, nose_angle, width, r_top, name = [p[x] for x in
                                                 relevant_params]

nose_height = 1.09 * strain * l ** 2 / th
nose_x = nose_height / math.tan(nose_angle)

# Step 2: Draw straight lines
point_data = ([(0, 0),  # Starts at bottom
           (l * 1.20, 1 / 2 * th * 1.25),
           (l * 1.20, 3 / 4 * th),
           (l * 1.07, th + nose_height),
           (l, th + nose_height),
           (l - nose_x, th),
           (r_top, th)
           ])
x = [point[0] for point in point_data]
y = [point[1] for point in point_data]

part = (
    cq.Workplane("XY")
        .moveTo(x[0], y[0])
)
for i in range(1, len(x)):
    part = part.lineTo(x[i], y[i])

# Step 3: Create a radius arc
# An arc is drawn from p0 to p2
# To get the correct arc, an additional point (p1) must be identified
p0 = (x[-1], y[-1])
p2 = (0, th + r_top)  # Top of radius

deg = math.radians(10)  # An arbitrary angle on the arc
p1_x = r_top * (1 - math.sin(deg))
p1_y = r_top * (1 - math.cos(deg)) + th
p1 = (p1_x, p1_y)
part = part.threePointArc(p1, p2)

# Step 4: close the curve and extrude
part = part.close()
part = part.extrude(width / 2, both=True)

# Step 4: Position part so that origin is in the intended center of part
part = part.translate((0, -th / 2, 0))

cant_part = part

In [36]:
# Create subtractive part

BASE_PARAMETERS = {}
BASE_PARAMETERS["thickness"] = 4
BASE_PARAMETERS["width"] = 8
BASE_PARAMETERS["length"] = 12
BASE_PARAMETERS["strain"] = 0.05
BASE_PARAMETERS["nose_angle"] = math.radians(60)
BASE_PARAMETERS["name"] = "default_cantilever"
BASE_PARAMETERS["r_top"] = 1.5
BASE_PARAMETERS["gap_length"] = 0.2
BASE_PARAMETERS["gap_thickness"] = 0.2
BASE_PARAMETERS["gap_width"] = 0.2
BASE_PARAMETERS["gap_legth"] = 0.15
BASE_PARAMETERS["extra_length"] = 0.5



# Step 1: Set variables to parameters
p = BASE_PARAMETERS.copy()


relevant_params = ["thickness", "length", "strain", "nose_angle",
                   "width", "r_top", "name"]
gap_params = ["gap_length", "gap_thickness", "gap_width", "extra_length", "gap_length"]

th, l, strain, nose_angle, width, r_top, name = [p[x] for x in
                                                 relevant_params]


gap_l, gap_th, gap_width, extra_length, gap_length = [p[x] for x in gap_params]


nose_height = 1.09 * strain * l ** 2 / th
nose_x = nose_height / math.tan(nose_angle)

# Step 2: Draw straight lines
point_data = ([(0, -gap_th),  # Starts at bottom
           (l * 1.20 + extra_length, 1 / 2 * th * 1.25 - gap_th),
           (l * 1.20 + extra_length, 3 / 4 * th),
           (l * 1.07 + extra_length, th + nose_height + gap_th),
           (l - gap_length, th + nose_height + gap_th),
           (l - nose_x - gap_length, th + gap_th),
           (r_top, th + gap_th),
               (0, th + r_top + gap_th)
           ])
x = [point[0] for point in point_data]
y = [point[1] for point in point_data]

part = (
    cq.Workplane("XY")
        .moveTo(x[0], y[0])
)
for i in range(1, len(x)):
    part = part.lineTo(x[i], y[i])

# Step 4: close the curve and extrude
part = part.close()
part = part.extrude(width / 2 - gap_width, both=True)

# Step 4: Position part so that origin is in the intended center of part
part = part.translate((0, -th / 2, 0))

assy = cq.Assembly()
assy.add(part)
assy.add(cant_part)
assy